In [ ]:
import os
import petl as etl
import pandas as pd
import magic
import sys
import csv
from openpyxl import Workbook
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.styles import PatternFill, Border, Side, Alignment, Protection, Font, NamedStyle

In [ ]:
maxInt = sys.maxsize
while True:
    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [ ]:
#Demander le nom du fichier de données
filename = input(str('Entrez le nom du fichier'))
# dans ce cas d'exemple: PLES_ExtractionTest_20210607

In [ ]:
#Demander le type du fichier
filetype = input(str('Type de fichier [M]ARC21, [c]sv, [t]exte, [x]ml:     M,C,T,X ? ')) 

In [ ]:
#Demander le séparateur
sep = input(str('Séparateur de champs à utiliser? '))

In [ ]:
#stocker le contenu du fichier
f = os.listdir("racine/source/test/" + filename)

In [ ]:
s = []
if filetype == "C":
    for i in range (0,len(f)):
        s.append(etl.fromcsv('racine/source/test/' + filename + '/' + f[i],delimiter = sep, encoding="utf8"))
elif filetype == "X":
    for i in range (0,len(f)):
        s.append(etl.fromxml('racine/source/test/' + filename + '/' + f[i], encoding="utf8"))
elif filetype == "T":
     for i in range (0,len(f)):
        s.append(etl.fromtext('racine/source/test/' + filename + '/' + f[i], encoding="utf8"))

In [ ]:
#generating the file
wb = Workbook()
#Generating the first sheet
ws = wb.active
ws.title = ("Fichiers")
#Les deux premieres lignes standard
Hd0 = ["source","Portfolio"]
Hd1 = ["Fichiers","Type","Separateur","Encodage","Quantite de donnees","Donnees","Cle du fichier","Fichier a fusionner","Cle a utiliser"]
ws.append(Hd0)
ws.append(Hd1)
#1ere colonne "fichier" 
for x in range(0,len(f)) :
    for row in ws.iter_rows(min_row=x+3, min_col=1, max_col=1, max_row=x+3):
        for cell in row:
            cell.value = f[x]
x+=1    
#2eme colonne "type"
for x in range(0,len(f)) :
    for row in ws.iter_rows(min_row=x+3, min_col=2, max_col=2, max_row=x+3):
        for cell in row:
            cell.value = magic.from_buffer(f[x])
x+=1
#3eme colonne "separateur"
for x in range(0,len(f)) :
    for row in ws.iter_rows(min_row=x+3, min_col=3, max_col=3, max_row=x+3):
        for cell in row:
            cell.value = sep
x+=1
#4eme colonne "encodage"
for x in range(0,len(f)) :
    for row in ws.iter_rows(min_row=x+3, min_col=4, max_col=4, max_row=x+3):
        for cell in row:
            cell.value = 'utf-8-'
x+=1
#5eme colonne "quantité de données"
for x in range(0,len(f)) :
    for row in ws.iter_rows(min_row=x+3, min_col=5, max_col=5, max_row=x+3):
        for cell in row:
            cell.value = len(s[x])
x+=1
#saving the file
wb.save("racine/sortie/test/profil.xlsx")

In [ ]:
#Generating the 2 & 3 sheet
ws2 = wb.create_sheet("Valeurs par defaut")
ws3 = wb.create_sheet("Correspondances")

ws2['A1'] = "Code régional(telephone)"
ws2['A2'] = "Province"

wb.save("racine/sortie/test/profil.xlsx")

In [ ]:
#Generating the other sheets
for i in range (0,len(f)):
    new_ws = wb.create_sheet("ws "+f[i])
wb.save("racine/sortie/test/profil.xlsx")

In [ ]:
#Remplissage des sheets créées 
Hd2 = ["Champs","Cle","Nombres de valeurs distinctes","Nombres de valeurs vides","Types de donnees","Champ Koha","Correspondance","Operateur","Valeur1","Valeur2"]

for x in range(3,len(wb.sheetnames)):
    wb.active = x
    wb.active.append(Hd2)
wb.save("racine/sortie/test/profil.xlsx")

In [ ]:
#La premiere ligne
for x in range(3,len(wb.sheetnames)):
    wb.active = x
    for i in range (0,len(s)):
        for j in range (0,len(etl.header(s[i]))):
            for row in wb.active.iter_rows(min_row=2, min_col=1, max_col=1):
                for cell in row:
                    cell.value = etl.header(s[i])[j]
wb.save("racine/sortie/test/profil.xlsx")

In [ ]:
#Nombre de valeurs distinctes
for x in range(3,len(wb.sheetnames)):
    wb.active = x
    for i in range (0,len(f)):
        c = list(etl.header(s[i]))
        for j in range (0,len(c)):
            val_dis = list(etl.valuecounter(s[i],c[j]))
            for row in wb.active.iter_rows(min_row=2, min_col=3, max_col=3, max_row=11):
                for cell in row:
                    cell = val_dis[j]
wb.save("racine/sortie/test/profil.xlsx")

In [ ]:
#Nombre de valeurs vides
for x in range(3,len(wb.sheetnames)):
    wb.active = x
    for i in range (0,len(f)):
        c = list(etl.header(s[i]))
        for j in range (0,len(c)):
            val_mis = list(etl.valuecounter(s[i],c[j]))
            for row in wb.active.iter_rows(min_row=2, min_col=4, max_col=4, max_row=11):
                for cell in row:
                    cell = val_mis[j]
wb.save("racine/sortie/test/profil.xlsx")